In [ ]:
import webbrowser
import time
import sys, os
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from tensorboard.plugins import projector

def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index_to_key), 100))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index_to_key):
            placeholder[i] = model.wv[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    tf.compat.v1.disable_eager_execution()
    sess =  tf.compat.v1.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    #tf.global_variables_initializer().run()
    tf.compat.v1.global_variables_initializer().run()
    saver =tf.compat.v1.train.Saver()
    #writer = tf.summary.create_file_writer(output_path, sess.graph)
    writer = tf.compat.v1.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file
    # 获取当前文件路径
    current_path = os.path.abspath('__file__')
    # 获取当前文件的父目录
    father_path = os.path.abspath(os.path.dirname(current_path) + os.path.sep + ".")
    # config.ini文件路径,获取当前目录的父目录的父目录与congig.ini拼接
    #config_file_path=os.path.join(os.path.abspath(os.path.dirname(current_path) + os.path.sep + ".."),'test')
    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    # print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))
    print('Run `tensorboard --logdir='+father_path+"\\visual` "+'to run visualize result on tensorboard')
    
    
if __name__ == "__main__":
    model = Word2Vec.load("./model/test.model")
    visualize(model,"./visual")

